### This notebook is for NTS and the summit
### We get the LUT force info from the EFD
### we compare the M2 simulator LUT implementation with Harris csv files, before and after the slew

In [ ]:
from openpyxl import load_workbook

from astropy.time import Time
from datetime import timedelta, datetime
from lsst_efd_client import EfdClient

import matplotlib.pyplot as plt
import scipy.io
import numpy as np
import pandas as pd
import os

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [ ]:
if summit:
    #start = Time('2021-03-25T15:10:00', scale = 'tai') #this is UTC
    #end   = Time('2021-03-25T15:11:00', scale = 'tai') 
    start = Time('2021-04-20T15:49:00', scale = 'tai') #this is UTC
    end   = Time('2021-04-20T15:50:00', scale = 'tai') 
    client = EfdClient('summit_efd')
else:
    #NCSA
    start = Time('2021-03-23T22:25:00', scale = 'tai') 
    end   = Time('2021-03-23T22:26:00', scale = 'tai') 
    client = EfdClient('ncsa_teststand_efd')

csc_index = 1
aF = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
tF = await client.select_time_series('lsst.sal.MTM2.tangentForce', '*', start, end, csc_index)
temp = await client.select_time_series('lsst.sal.MTM2.temperature', '*', start, end, csc_index)
zA = await client.select_time_series('lsst.sal.MTM2.zenithAngle', '*', start, end, csc_index)
#for the testing on 4/20/21, we always used mount telemetry for M2 LUT - so the below is needed
#zA = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
#zA['measured'] = 90 - zA.actualPosition

In [ ]:
if summit:
    #start = Time('2021-03-25T16:10:00', scale = 'tai') #this is UTC
    #end   = Time('2021-03-25T16:11:00', scale = 'tai') 
    start = Time('2021-04-20T15:51:30', scale = 'tai') #this is UTC
    end   = Time('2021-04-20T15:55:00', scale = 'tai') 
    client = EfdClient('summit_efd')
else:
    #NCSA
    start = Time('2021-03-23T22:29:00', scale = 'tai') 
    end   = Time('2021-03-23T22:30:00', scale = 'tai') 
    client = EfdClient('ncsa_teststand_efd')

aF2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
tF2 = await client.select_time_series('lsst.sal.MTM2.tangentForce', '*', start, end, csc_index)
temp2 = await client.select_time_series('lsst.sal.MTM2.temperature', '*', start, end, csc_index)
if summit:
    zA2 = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
    zA2['measured'] = 90 - zA2.actualPosition
else:
    zA2 = await client.select_time_series('lsst.sal.MTM2.zenithAngle', '*', start, end, csc_index)

In [ ]:
len(aF), len(aF2)

### do things change with time, for example, LUT gravity?

In [ ]:
x01 = zA.private_sndStamp - zA.private_sndStamp[0]
x02 = zA2.private_sndStamp - zA2.private_sndStamp[0]
fig, ax = plt.subplots(figsize=(15,3))
plt.plot(x01, 90-zA.measured, label='before elevation change')
plt.plot(x02, 90-zA2.measured, label='after elevation change')
plt.title('elevation ')
plt.xlabel('time (in seconds)')
plt.legend()
plt.grid();

In [ ]:
x02 = aF2.private_sndStamp - aF2.private_sndStamp[0]
fig, ax = plt.subplots(figsize=(15,3))
i = 0
plt.plot(x01, aF['lutGravity%d'%i], label='before elevation change')
plt.plot(x02, aF2['lutGravity%d'%i], label='after elevation change')
plt.title('gravity LUT force on first actuator')
plt.xlabel('time (in seconds)')
plt.legend()
plt.grid();

In [ ]:
#plot temperatures before and after
fig, ax = plt.subplots(figsize=(15,3))
plt.plot([temp['ring%d'%i][0] for i in range(12)], label='ring')
plt.plot([temp.intake0[0], temp.intake1[0]],'-o', label = 'intake')
plt.plot([temp.exhaust0[0], temp.exhaust1[0]],'-*', label = 'exhaust')
plt.plot([temp2['ring%d'%i][0] for i in range(12)], label='ring 2')
plt.plot([temp2.intake0[0], temp.intake1[0]],'-o', label = 'intake 2')
plt.plot([temp2.exhaust0[0], temp.exhaust1[0]],'-*', label = 'exhaust 2')
plt.title('temperature')
plt.legend()
plt.grid() 

#### FinalHandingLUTs vs FinalOpticalLUTs 
* https://github.com/lsst-ts/ts_mtm2_cell/tree/master/configuration/lsst-m2/config/parameter_files/luts
* There are 8 files in each subfolder: FinalHandingLUTs vs FinalOpticalLUTs. The only file that is different is F_F.csv
* This is easily understandable, because the optical LUT cares about mirror shape at any zenith angle. While the other one is used for on-cart rotation.
* Which one did we use for the 2020 Feb summit test? Te-Wei says we used HandlingLUTs.
* Do we know how to switch? Yes. By the high level configuration file.

In [ ]:
LUTfolder = '%s/notebooks/M2_summit_2003/harrisLUT/FinalHandlingLUTs'%(os.environ["HOME"])
dfe = pd.read_csv('%s/F_E.csv'%LUTfolder)
FE = np.float64(dfe)
df0 = pd.read_csv('%s/F_0.csv'%LUTfolder)
F0 = np.float64(df0)
dff = pd.read_csv('%s/F_F.csv'%LUTfolder)
FF = np.float64(dff)
dfa = pd.read_csv('%s/F_A.csv'%LUTfolder)
FA = np.float64(dfa)
elev = np.float64(dfe.keys()) # the columns are labelled with elevation angle.
dtr = pd.read_csv('%s/Tr.csv'%LUTfolder, header=None)
Tr = np.float64(dtr)
dtu = pd.read_csv('%s/Tu.csv'%LUTfolder, header=None)
Tu = np.float64(dtu)
dtx = pd.read_csv('%s/Tx.csv'%LUTfolder, header=None)
Tx = np.float64(dtx)
dty = pd.read_csv('%s/Ty.csv'%LUTfolder, header=None)
Ty = np.float64(dty)

LUTfolder = '%s/notebooks/M2_summit_2003/harrisLUT/FinalOpticalLUTs'%(os.environ["HOME"])
dff = pd.read_csv('%s/F_F.csv'%LUTfolder)
FF1 = np.float64(dff)

In [ ]:
M2Weight = 15578 #Newton
def lookUpGravity(summit, lutInAngle, FE, FA,elevA):
    '''
    input:
        lutInAngle is the angle used to define the LUT, which is the elevation angle.
        elevA is the actual measured elevation angle.
    output:
        gravity LUT force
    '''
    myfe = np.zeros(78)
    myfa = np.zeros(78)
    myf0 = np.zeros(78)
    myff = np.zeros(78)
    for i in range(72):
        myfe[i] = np.interp(elevA, lutInAngle, FE[i,:])
        myf0[i] = np.interp(elevA, lutInAngle, F0[i,:])
        myff[i] = np.interp(elevA, lutInAngle, FF[i,:])
    F2 = M2Weight* np.sin(np.radians(90-elevA))/2/np.sqrt(3)
    myfe[73] = F2
    myfe[74] = F2
    myfe[76] = -F2
    myfe[77] = -F2
    for i in range(78):
        myfa[i] = np.interp(elevA, lutInAngle, FA[i,:])
    myf = myfe + myfa 
    if summit: 
        #with TeWei's summit implementation, we also included f0 and ff in lutGravity
        myf = myf + myf0 + myff
    else:
        #The tangent forces in the M2 simulator has a known bug (DM-28515) - all the tangent force should have reversed sign.
        myfe[72:] = -myfe[72:]
    return myf

def lookUpTemperature(Tr, Tx, Ty, Tu, tempInv, efdT):
    '''
    input:
        efdT is the list of temperature values ordered as [0-11, intake0, intake1, exhaust0, exhaust1]
    output:
        thermal LUT force
    '''
    #Order temperature data based on a12_temperature.ipynb
    binT = efdT[[0,1,2,3, 12,15,14,13, 8,9,10,11, 4,5,6,7]]
    lutT = binT[ [1, 2, 3, 12, 9, 8, 13, 14, 15, 11, 10, 0]]
    tref = 21 #deg C. 
    tcoef = tempInv.dot(lutT-tref)
    
    for i in range(72):
        myft = tcoef[0]*Tr + tcoef[1]*Tx + tcoef[2]*Ty + tcoef[3]*Tu
    myf = np.squeeze(myft)
    return myf

In [ ]:
zA.measured[0]

In [ ]:
zA2.measured[0]

In [ ]:
#check lutGravity against expectation
myFEa = [aF['lutGravity%d'%i][0] for i in range(72)] 
myFEt = [tF['lutGravity%d'%i][0] for i in range(6)]
myFEa2 = [aF2['lutGravity%d'%i][0] for i in range(72)] 
myFEt2 = [tF2['lutGravity%d'%i][0] for i in range(6)]
Felev = lookUpGravity(summit, elev, FE, FA, 90-zA.measured[0])
Felev2 = lookUpGravity(summit, elev, FE, FA, 90-zA2.measured[0])
fig, ax = plt.subplots(2,1, sharex = False, figsize=(15,6))
ax[0].plot(myFEa ,'bo', label='EFD 1')
ax[0].plot(Felev[:72], label='expected 1')
ax[0].plot(myFEa2 ,'ro', label='EFD 2')
ax[0].plot(Felev2[:72], '-', label='expected 2')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Axial')
ax[1].plot(myFEt,'b*', markersize= 15, label='EFD 1')
ax[1].plot(Felev[72:], label='expected 1')
ax[1].plot(myFEt2,'ro', label='EFD 2')
ax[1].plot(Felev2[72:], '-', label='expected 2')
ax[1].grid()
#ax[1].set_ylim([-1,1])
ax[1].legend()
ax[1].set_title('Tangent')
plt.suptitle('LUT Gravity');
#The tangent forces in the M2 simulator has a known bug (DM-28515) - all the tangent force should have reversed sign.

In [ ]:
tempInv = np.loadtxt('%s/notebooks/M2_summit_2003/harrisLUT/temp_inv.txt'%(os.environ["HOME"]))

### thermal LUT

In [ ]:
#The intake and exhaust temps are not use for LUT
tt = [temp['ring%d'%i][0] for i in range(4)] + [0,0,0,0] + \
    [temp['ring%d'%i][0] for i in range(8,12)] + [temp['ring%d'%i][0] for i in range(4,8)]
idx = [1, 2, 3, 12, 9, 8, 13, 14, 15, 11, 10, 0]
myt= np.array([tt[i] for i in idx])

tref = 21 #deg C. 
tcoef = tempInv.dot(myt-tref)
FT = tcoef[0]*Tr + tcoef[1]*Tx + tcoef[2]*Ty + tcoef[3]*Tu

tt = [temp2['ring%d'%i][0] for i in range(4)] + [0,0,0,0] + \
    [temp2['ring%d'%i][0] for i in range(8,12)] + [temp2['ring%d'%i][0] for i in range(4,8)]
myt= np.array([tt[i] for i in idx])

tcoef = tempInv.dot(myt-tref)
FT2 = tcoef[0]*Tr + tcoef[1]*Tx + tcoef[2]*Ty + tcoef[3]*Tu

In [ ]:
#check lutTemperature against expectation
myFTa = [aF['lutTemperature%d'%i][0] for i in range(72)] 
myFTt = [tF['lutTemperature%d'%i][0] for i in range(6)]
myFTa2 = [aF2['lutTemperature%d'%i][0] for i in range(72)] 
myFTt2 = [tF2['lutTemperature%d'%i][0] for i in range(6)]
fig, ax = plt.subplots(2,1, sharex = False, figsize=(15,6))
ax[0].plot(myFTa ,'ro', label='EFD')
ax[0].plot(FT[:,0], label='expected')
ax[0].plot(myFTa2 ,'ro', label='EFD')
ax[0].plot(FT2[:,0], '-.', label='expected')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Axial')
ax[1].plot(myFTt,'ro', label='EFD')
ax[1].plot([0]*6, label='expected')
ax[1].plot(myFTt2,'ro', label='EFD')
ax[1].plot([0]*6, '-.', label='expected')
ax[1].grid()
#ax[1].set_ylim([-1,1])
ax[1].legend()
ax[1].set_title('Tangent')
plt.suptitle('LUT Temperature');

In [ ]:
plt.plot([aF['hardpointCorrection%d'%i][0] for i in range(72)], label='before elevation change')
plt.plot([aF2['hardpointCorrection%d'%i][0] for i in range(72)], label='after elevation change')
plt.legend();